In [1]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name

bucket='sagemaker-linkage' # Replace with your s3 bucket name
prefix = 'sagemaker/xgboost-mnist' # Used as part of the path in the bucket where you store data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket) # The URL to access the bucket

In [2]:
!pip install recordlinkage
import recordlinkage

You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

In [4]:
from recordlinkage.datasets import load_febrl4
#public datasets from recordlinkage libary

In [5]:
df_a, df_b = load_febrl4()
df_a

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548
rec-1985-org,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555
rec-1473-org,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058


In [6]:
df_b

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
rec-4285-dup-0,sophie,manson,14,elizabeth xrescent,manorhouse,gorokan,3465,vic,19510201,4818868
rec-929-dup-0,madison,morrison,38,peacock place,melim a,campbelltown,3021,wa,19481214,1967265
rec-4833-dup-0,jayed,millar,6,meredith c ircuit,NaN,thornbury,6014,vic,19340731,1314310
rec-717-dup-0,joshua,raward,26,yambina cerscent,thorndale,macgrdgor,4718,qld,NaN,9086686


In [7]:
indexer = recordlinkage.SortedNeighbourhoodIndex(left_on = 'surname', right_on = 'surname', window = 3)
links = indexer.index(df_a,df_b)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/recordlinkage/utils.py:47: VisibleDeprecationWarning: class recordlinkage.SortedNeighbourhoodIndex is renamed and moved, use recordlinkage.index.SortedNeighbourhood
  warn(msg, VisibleDeprecationWarning, stacklevel=1)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/recordlinkage/index.py:348: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  verify_integrity=False)


In [8]:
print(len(links))

97699


In [9]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()

compare_cl.string('given_name', 'given_name', method = 'jarowinkler', threshold = 0.85, label = 'name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')
compare_cl.exact('soc_sec_id', 'soc_sec_id', label = 'ss_id')

features = compare_cl.compute(links, df_a, df_b)
features

name  surname  date_of_birth  suburb  state  \
rec_id_1     rec_id_2                                                      
rec-1070-org rec-2820-dup-0   0.0      0.0              0       0      1   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-2797-org rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      1   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-4388-org rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      1   
rec-4387-org rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      1   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-2672-org rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      1   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-2158-org rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      1   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      0   
rec-787-org  rec-2820-dup-0   0.0      0.0              0       0      0   
             rec-4348-dup-0   0.0      0.0              0       0      0   
             rec-2276-dup-0   0.0      0.0              0       0      0   
             rec-3382-dup-0   0.0      0.0              0       0      1   
rec-1016-org rec-264-dup-0    0.0      1.0              0       0      0   
rec-355-org  rec-264-dup-0    0.0      1.0              0       0      1   
...                           ...      ...            ...     ...    ...   
rec-3830-org rec-339-dup-0    0.0      0.0              0       0      0   
             rec-4206-dup-0   0.0      0.0              0       0      0   
             rec-2195-dup-0   0.0      0.0              0       0      1   
             rec-1381-dup-0   0.0      0.0              0       0      1   
rec-1116-org rec-2100-dup-0   0.0      1.0              0       0      0   
rec-1256-org rec-4328-dup-0   0.0      0.0              0       0      0   
rec-3072-org rec-4137-dup-0   0.0      0.0              0       0      0   
rec-2177-org rec-2896-dup-0   0.0      0.0              0       0      0   
             rec-4744-dup-0   0.0      0.0              0       0      0   
rec-3380-org rec-2342-dup-0   0.0      0.0              0       0      0   
             rec-3794-dup-0   0.0      0.0              0       0      0   
             rec-1060-dup-0   0.0      0.0              0       0      0   
             rec-3891-dup-0   0.0      0.0              0       0      0   
rec-2869-org rec-2342-dup-0   0.0      0.0              0       0      0   
             rec-3794-dup-0   0.0      0.0              0       0      1   
             rec-1060-dup-0   0.0      0.0              0       0      0   
             rec-3891-dup-0   0.0      0.0              0       0      0   
rec-974-org  rec-3756-dup-0   0.0      0.0              0       0      1   
rec-1402-org rec-1402-dup-0   1.0      1.0              1       0      1   
rec-2606-org rec-1185-dup-0   0.0      0.0              0       0      0   
r

In [10]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

7.0     1514
6.0     1660
5.0      554
4.0       95
3.0      159
2.0    19703
1.0    69886
0.0     4128
dtype: int64

In [19]:
goldenpairs = features[features.sum(axis=1) > 5]
goldenpairs
#all the pairs with more than 3 one's in the features, simple but not effective

,,name,surname,date_of_birth,suburb,state,address_1,ss_id
rec_id_1,rec_id_2,,,,,,,
rec-4312-org,rec-4312-dup-0,1.0,1.0,1,0,1,1.0,1
rec-4372-org,rec-4372-dup-0,1.0,1.0,1,1,1,1.0,1
rec-170-org,rec-170-dup-0,1.0,1.0,1,0,1,1.0,1
rec-1991-org,rec-1991-dup-0,1.0,1.0,1,1,1,1.0,1
rec-2389-org,rec-2389-dup-0,1.0,1.0,1,1,1,1.0,0
rec-3-org,rec-3-dup-0,1.0,1.0,1,1,1,1.0,1
rec-1826-org,rec-1826-dup-0,1.0,1.0,0,1,1,1.0,1
rec-204-org,rec-204-dup-0,1.0,1.0,1,0,1,1.0,1
rec-3998-org,rec-3998-dup-0,1.0,1.0,1,1,1,1.0,1


In [25]:
golden_tuple = list(goldenpairs.index.values)

In [26]:
golden_tuple_df = pd.DataFrame(golden_tuple, columns=['a', 'b'])
golden_tuple_df

,a,b
0,rec-4312-org,rec-4312-dup-0
1,rec-4372-org,rec-4372-dup-0
2,rec-170-org,rec-170-dup-0
3,rec-1991-org,rec-1991-dup-0
4,rec-2389-org,rec-2389-dup-0
5,rec-3-org,rec-3-dup-0
6,rec-1826-org,rec-1826-dup-0
7,rec-204-org,rec-204-dup-0
8,rec-3998-org,rec-3998-dup-0
9,rec-3400-org,rec-3400-dup-0


In [31]:
tupleDict = dict(golden_tuple)

tupleDict

{'rec-4312-org': 'rec-4312-dup-0',
 'rec-4372-org': 'rec-4372-dup-0',
 'rec-170-org': 'rec-170-dup-0',
 'rec-1991-org': 'rec-1991-dup-0',
 'rec-2389-org': 'rec-2389-dup-0',
 'rec-3-org': 'rec-3-dup-0',
 'rec-1826-org': 'rec-1826-dup-0',
 'rec-204-org': 'rec-204-dup-0',
 'rec-3998-org': 'rec-3998-dup-0',
 'rec-3400-org': 'rec-3400-dup-0',
 'rec-3054-org': 'rec-3054-dup-0',
 'rec-1271-org': 'rec-1271-dup-0',
 'rec-3556-org': 'rec-3556-dup-0',
 'rec-4659-org': 'rec-4659-dup-0',
 'rec-1185-org': 'rec-1185-dup-0',
 'rec-3815-org': 'rec-3815-dup-0',
 'rec-2495-org': 'rec-2495-dup-0',
 'rec-257-org': 'rec-257-dup-0',
 'rec-3020-org': 'rec-3020-dup-0',
 'rec-3721-org': 'rec-3721-dup-0',
 'rec-3817-org': 'rec-3817-dup-0',
 'rec-3327-org': 'rec-3327-dup-0',
 'rec-49-org': 'rec-49-dup-0',
 'rec-451-org': 'rec-451-dup-0',
 'rec-1819-org': 'rec-1819-dup-0',
 'rec-1434-org': 'rec-1434-dup-0',
 'rec-4722-org': 'rec-4722-dup-0',
 'rec-816-org': 'rec-816-dup-0',
 'rec-2356-org': 'rec-2356-dup-0',
 'rec

In [61]:
# df_a['pair'] = np.empty((len(df_a), 0)).tolist()
# df_a['pair'] = ""
# df_a
#add an empty list to each column of dataframe a to store the possible linkages

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,pair
rec_id,,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218,
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548,
rec-1985-org,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690,
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555,
rec-1473-org,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058,


In [62]:
df_a.dtypes

given_name       object
surname          object
street_number    object
address_1        object
address_2        object
suburb           object
postcode         object
state            object
date_of_birth    object
soc_sec_id       object
pair             object
dtype: object

In [67]:
for index, row in df_a.iterrows():
#         print(index + ':' + len(df_a.at[index,'pair']))
        val = tupleDict.get(index)
        df_a.at[index,'pair'] = val
        
df_a

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,pair
rec_id,,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218,None
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,rec-1016-dup-0
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,rec-4405-dup-0
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,rec-1288-dup-0
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,rec-3585-dup-0
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548,rec-298-dup-0
rec-1985-org,NaN,lund,109,caley crescent,allandale aged care facility,mill park,4053,nsw,19180902,7074690,None
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555,rec-2404-dup-0
rec-1473-org,NaN,leslie,925,carpenter close,NaN,canterbury,2340,vic,19950608,2438058,None


In [70]:
df_a.dropna(subset=['pair'], inplace=True)
df_a

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,pair
rec_id,,,,,,,,,,,
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625,rec-1016-dup-0
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168,rec-4405-dup-0
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102,rec-1288-dup-0
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688,rec-3585-dup-0
rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548,rec-298-dup-0
rec-2404-org,blakeston,broadby,53,traeger street,valley of springs,north ward,3083,qld,19120907,4308555,rec-2404-dup-0
rec-453-org,edward,denholm,10,corin place,gold tyne,clayfield,4221,vic,19660306,7119771,rec-453-dup-0
rec-4866-org,charlie,alderson,266,hawkesbury crescent,deergarden caravn park,cooma,4128,vic,19440908,1256748,rec-4866-dup-0
rec-4302-org,molly,roche,59,willoughby crescent,donna valley,carrara,4825,nsw,19200712,1847058,rec-4302-dup-0
